# Web Scraper IMDB Movies

## Import Libraries

In [1]:
import pandas as pd
import numpy as np

import requests
from requests import get
from bs4 import BeautifulSoup

from time import sleep
from random import randint

## Scraping from the web page

In [2]:
# Creating the lists we want to write into
titles = []
years = []
time = []
imdb_ratings = []
metascores = []
votes = []
us_gross = []

In [3]:
headers = {'Accept-Language': 'en-US, en;q=0.5'}

In [4]:
pages = np.arange(1, 1001, 50)
pages

array([  1,  51, 101, 151, 201, 251, 301, 351, 401, 451, 501, 551, 601,
       651, 701, 751, 801, 851, 901, 951])

In [5]:
# Storing each of the urls of 50 movies 
for page in pages:
    # Getting the contents from the each url
    page = requests.get('https://www.imdb.com/search/title/?groups=top_1000&start=' + str(page) + '&ref_=adv_nxt', headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # Aiming the part of the html we want to get the information from
    movie_div = soup.find_all('div', class_='lister-item mode-advanced')
    
    # Controling the loop’s rate by pausing the execution of the loop for a specified amount of time
    # Waiting time between requests for a number between 2-10 seconds
    sleep(randint(2,10))
    
    for container in movie_div:
        # Scraping the movie's name
        name = container.h3.a.text
        titles.append(name)
        
        # Scraping the movie's year
        year = container.h3.find('span', class_='lister-item-year').text
        years.append(year)
        
        # Scraping the movie's length
        runtime = container.find('span', class_='runtime').text if container.p.find('span', class_='runtime') else '-'
        time.append(runtime)
        
        # Scraping the rating
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)
        
        # Scraping the metascore
        m_score = container.find('span', class_='metascore').text if container.find('span', class_='metascore') else '-'
        metascores.append(m_score)
        
        # Scraping votes and gross earnings
        nv = container.find_all('span', attrs={'name':'nv'})
        vote = nv[0].text
        votes.append(vote)
        grosses = nv[1].text if len(nv) > 1 else '-'
        us_gross.append(grosses)

## Creating the Data Set

In [6]:
movies = pd.DataFrame({'movie':titles,
                       'year':years,
                       'time_minute':time,
                       'imdb_rating':imdb_ratings,
                       'metascore':metascores,
                       'vote':votes,
                       'gross_earning':us_gross})

movies.head()

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Dune,(2021),155 min,8.4,75,"69,747",-
1,Shang-Chi and the Legend of the Ten Rings,(2021),132 min,7.9,71,"103,095",-
2,Casino Royale,(2006),144 min,8.0,80,"607,941",$167.45M
3,Skyfall,(2012),143 min,7.8,81,"654,074",$304.36M
4,Knives Out,(2019),130 min,7.9,82,"533,271",$165.36M


## Cleaning the Data Set

In [7]:
# Cleaning 'year' column
movies['year'] = movies['year'].str.extract('(\d+)').astype(int)
movies.head(3)

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Dune,2021,155 min,8.4,75,"69,747",-
1,Shang-Chi and the Legend of the Ten Rings,2021,132 min,7.9,71,"103,095",-
2,Casino Royale,2006,144 min,8.0,80,"607,941",$167.45M


In [10]:
movies.tail(3)

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
997,English Vinglish,2012,134,7.8,-,"35,056",$1.67M
998,The Breath,2009,128,8.0,-,"32,838",-
999,Andaz Apna Apna,1994,160,8.1,-,"51,761",-


In [8]:
movies.dtypes

movie             object
year               int32
time_minute       object
imdb_rating      float64
metascore         object
vote              object
gross_earning     object
dtype: object

In [9]:
# Cleaning 'time_minute' column
movies['time_minute'] = movies['time_minute'].str.extract('(\d+)').astype(int)
movies.head(3)

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Dune,2021,155,8.4,75,"69,747",-
1,Shang-Chi and the Legend of the Ten Rings,2021,132,7.9,71,"103,095",-
2,Casino Royale,2006,144,8.0,80,"607,941",$167.45M


In [11]:
# Cleaning 'metascore' column
movies['metascore'] = movies['metascore'].str.extract('(\d+)')
# convert it to float and if there are dashes turn it into NaN
movies['metascore'] = pd.to_numeric(movies['metascore'], errors='coerce')

In [12]:
# Cleaning 'vote' column
movies['vote'] = movies['vote'].str.replace(',', '').astype(int)
movies.head(3)

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Dune,2021,155,8.4,75.0,69747,-
1,Shang-Chi and the Legend of the Ten Rings,2021,132,7.9,71.0,103095,-
2,Casino Royale,2006,144,8.0,80.0,607941,$167.45M


In [14]:
movies.tail(10)

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
990,Chak De! India,2007,153,8.2,68.0,77508,$1.11M
991,Munna Bhai M.B.B.S.,2003,156,8.1,NaN,78027,-
992,"Carry On, Munna Bhai",2006,144,8.1,NaN,45337,$2.22M
993,Anand,1971,122,8.2,NaN,32839,-
994,Paan Singh Tomar,2012,135,8.2,NaN,34964,$0.04M
995,Kai Po Che,2013,120,7.8,40.0,35388,$1.12M
996,Vizontele,2001,110,8.0,NaN,35266,-
997,English Vinglish,2012,134,7.8,NaN,35056,$1.67M
998,The Breath,2009,128,8.0,NaN,32838,-
999,Andaz Apna Apna,1994,160,8.1,NaN,51761,-


In [15]:
# Cleaning 'gross_earning' column
# left strip $ and right strip M 
movies['gross_earning'] = movies['gross_earning'].map(lambda x: x.lstrip('$').rstrip('M'))
# convert it to float and if there are dashes turn it into NaN
movies['gross_earning'] = pd.to_numeric(movies['gross_earning'], errors='coerce')
movies.head(3)

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Dune,2021,155,8.4,75.0,69747,NaN
1,Shang-Chi and the Legend of the Ten Rings,2021,132,7.9,71.0,103095,NaN
2,Casino Royale,2006,144,8.0,80.0,607941,167.45


## Final Data Set

In [16]:
movies

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Dune,2021,155,8.4,75.0,69747,NaN
1,Shang-Chi and the Legend of the Ten Rings,2021,132,7.9,71.0,103095,NaN
2,Casino Royale,2006,144,8.0,80.0,607941,167.45
3,Skyfall,2012,143,7.8,81.0,654074,304.36
4,Knives Out,2019,130,7.9,82.0,533271,165.36
...,...,...,...,...,...,...,...
995,Kai Po Che,2013,120,7.8,40.0,35388,1.12
996,Vizontele,2001,110,8.0,NaN,35266,NaN
997,English Vinglish,2012,134,7.8,NaN,35056,1.67
998,The Breath,2009,128,8.0,NaN,32838,NaN


In [17]:
movies.isnull().sum()

movie              0
year               0
time_minute        0
imdb_rating        0
metascore        162
vote               0
gross_earning    179
dtype: int64

## Saving the data to a csv file

In [20]:
movies.to_csv('IMDB_Movies.csv')